# Machine Learning workshop

In this workshop, we will study GSE53987 dataset on Bipolar disorder (BD) and major depressive disorder (MDD) and schizophrenia. You can download it [here](https://github.com/BRITE-REU/programming-workshops/blob/master/source/workshops/04_Machine_learning/data/GSE53987_combined.csv).

In total there are 205 rows consisting of 19 individuals diagnosed with BPD, 19 with MDD, 19 schizophrenia and 19 controls. Each sample has gene expression from 3 tissues (post-mortem brain). There are a total of 13768 genes (numeric features) and 10 meta features and 1 ID (GEO sample accession).

- Age
- Race (W for white and B for black)
- Gender is F for female and M for male
- Ph is the ph of the brain tissue
- Pmi is the post mortal interval
- Rin is the RNA integrity number
- Patient is unique for each patient. Each patient has up to 3 tissue samples. The patient ID is written as disease followed by a number from 1 to 19
- Tissue is the tissue the expression was obtained from.
- Disease.state is the class of disease the patient belongs to: bipolar, schizophrenia, depression or control.
- source.name is the combination of th etissue and disease.state

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data (change location if necessary)
data = pd.read_csv("../data/GSE53987_combined.csv", index_col=0)

In [ ]:
# Let's take a look at the first 5 rows of our dataframe
data.head()

## Task 1

Check all the features. Which features are numeric, which are categorical? Understanding the nature of your data is a very important and necessary first step before proceeding with any analysis.

- What type of distributions exist within the features? Is Gender a balanced feature (roughly equal representation between both men and women)? Are numerical values normally distributed? Explore numerical distributions by plotting histograms for Age, an Age + Gender histogram, and one of your favorite genes found in the dataset.

- Some features display factor dependent values. That is, whether a subject is a male or a female might effect the expression patterns of a given gene. Explore factor and feature relationships by creating boxplots to observe how Age is dependent on Tissue, Gender and Disease.status.

In [ ]:
# Explore the distribution of genders


In [ ]:
# Histogram of the age
sns.distplot(data["Age"])

In [ ]:
# Histogram of the Age faceted by Gender
g = sns.FacetGrid(data, col="Gender", margin_titles=True)
g.map(sns.distplot, "Age")

In [ ]:
# Histogram of the gene expression of ZWINT across samples


In [ ]:
# Relationship between age, disease state and gender
sns.catplot(data=data, x="Disease.state", y="Age", hue="Gender", kind="box")

In [ ]:
# # Relationship between age, disease state, tissue type and gender(facet)
sns.catplot(data=data, x="Disease.state", y="Age", hue="Tissue", col="Gender", kind="box")

## Task 2

Principal Component Analysis (PCA) is a commonly used technique to create linearly uncorrelated features from a set of possibly correlated features. The procedure is done in such a way that the first feature produced by PCA, the first principal component – PC1, explains the largest amount of variability possible. In this way, PCA is a dimension reduction technique, as the first few principal components often explain upwards of 90% of the variability found within a dataset. It is important to note that if we’re planning on predicting anything using the principal components, such as tissue type or Disease.status, those features should not be included in the input matrix. Before performing PCA, create a new data frame containing only explanatory values (i.e. the features we want to use to predict class membership).

- Explore how much variation is explained by the principal components. How much variation is explained by the first two principal components? How many principal components might be required to explain 75%, 85%, 90%, 95%, and 99% of the variation within our dataset?

- Visually explore this separation to plot the first two principal components and color samples according to Tissue and Disease.status. What effect does plotting the third principal component have on sample separation?

- Subset the dataset into three disjoint datasets by Tissue. Run PCA on all three of these datasets, plot the first two principal components, and color the dots according to Disease.status. Does there appear to be a meaningful difference in the separation between disease classes between the three different datasets?


In [ ]:
# Refer to: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.decomposition import PCA

In [ ]:
# Create a dataframe containing only the expression data (column 10 onwards)

# Create a PCA class with 2 components

# Fit and tranform the data


In [ ]:
# How much variance is explanined by the principal components?


In [ ]:
# Visualize the PCA plot color by Tissue or Disease.state
g = sns.scatterplot(x=expression_decomp[:, 0], y=expression_decomp[:, 1], hue=data["Tissue"])
g.set_xlabel("PCA1")
g.set_ylabel("PCA2")

In [ ]:
# Hippocampus subset
hippocampus = data[data["Tissue"] == "hippocampus"]
# Create a dataframe containing only the expression data

# Create a PCA class with 2 components

# Fit and tranform the data


In [ ]:
# Visualize the hippocampus PCA plot
g = sns.scatterplot(x=hippocampus_expression_decomp[:, 0], y=hippocampus_expression_decomp[:, 1], hue=hippocampus["Disease.state"])
g.set_xlabel("PCA1")
g.set_ylabel("PCA2")

## Task 3

Feature selection is a commonly performed step in statistics/machine learning to distinguish the most informative variable to use in model creation. There are several different ways to perform feature selection, and many of these can be application specific. In this workshop we’ll explain two possible avenues for feature selection in gene expression data analysis: 1) removing the least variable features 2) univariate feature selection

In [ ]:
# Refer to https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2

In [ ]:
# Removing features with low variance (<0.1)
sel = VarianceThreshold(threshold=0.1)
expression_highvar = sel.fit_transform(expression)

In [ ]:
# Univariate feature selection using the chi2 test
target = data["Tissue"]
expression_kbest = SelectKBest(chi2, k=10).fit_transform(expression, target)

## Task 4

Unsupervised learning can be thought of as applying an algorithm to a dataset in order to discover latent structure that exists between samples. We’ve already been exposed to some of these algorithms via PCA. However, one of the most common techniques in machine learning, and especially bioinformatics, is clustering. Cluster the data using the k-means algorithm.

In [ ]:
# Refer to https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
from sklearn.cluster import KMeans

In [ ]:
# Initialize a KMeans object with n_clusters=3

# Fit data and get predictions


In [ ]:
# Compare predicted tissue data to actual tissue data

In [ ]:
# Transform expression into a cluster-distance space and visualize the results using sns.scatterplot


## Task 5

Supervised learning is a technique to teach an algorithm to distinguish between previously labelled groups, such as Tissue, Gender, or Disease.status. However, all supervised methods require data to learn how to differentiate between classes. Therefore, it is necessary to separate data into test/train sets. The training set is used to train the model, while the test set is used to evaluate performance. Cross-validation, a method of partitioning the data into disjoint subsets and continually re-training and re-testing with different partition combinations, is often used to evaluate models. In this section, we will build various classifiers using logistic regression to predict different classes from our data. You should evaluate your models’ performances using confusion matrices and accuracy scores.

In [ ]:
# Refer to https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [ ]:
# Set the "target" of the classifier (either "Disease.state" or "Tissue")


In [ ]:
# Split the data into training and test subsets with test_size=.25 and stratify using target


In [ ]:
# Initialize the LogisticRegression class with solve="lbfgs" and multi_class="auto"

# Fit and predict


In [ ]:
# What is the accuracy score of the test data?


In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix


In [ ]:
# Refer to https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
from sklearn.model_selection import cross_val_score

In [ ]:
# Perform a 5 fold cross validation on the model


In [ ]:
# Cross validation results
